# Assignment 4

## Q1

In [1]:
import sys
!{sys.executable} -m pip install SimPyClassic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 2.8 MB/s eta 0:00:00


In [2]:
from SimPy.Simulation import *
import random
import numpy
import math

In [3]:
def tablelookup(P):
    """Sample from i=0..n-1 with probabilities P[i]"""
    u = random.random()
    sumP = 0.0
    for i in range(len(P)):
        sumP += P[i]
        if u<sumP:
            return i
        
def conf(L):
    """confidence interval"""
    lower = numpy.mean(L) - 1.96*numpy.std(L)/math.sqrt(len(L))
    upper = numpy.mean(L) + 1.96*numpy.std(L)/math.sqrt(len(L))
    return lower, upper

In [4]:
class Job(Process):
    """a job"""
    n = 0 # number in system
    
    def run(self, N):
        startTime = now()
        current = G.startNode
        
        Job.n += 1
        G.numberMon.observe(Job.n)
        
        for i in range(N):
            yield request, self, G.server[current]
            t = random.expovariate(G.mu[current])
            yield hold, self, t
            yield release, self, G.server[current]
            
            delay = now() - startTime
            G.delayMon[current].observe(delay)
            
            current = tablelookup(G.transition[current])
            startTime = now()
            
        Job.n -= 1
        G.numberMon.observe(Job.n)
            
class G:
    server = 'dummy'
    delayMon = 'Monitor'
    numberMon = 'Monitor'
    startNode = 0
    mu = [1.0, 0.1, 1/1.4]
    mu = [1/20, 1/10, 1/30]
    
    # let p = 0.4 => 1-p=0.6
    transition = [
        [0, 0.3, 0.7],
        [1, 0, 0],
        [0, 1, 0]
    ]
    
def model(server, N, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = [Resource(server),
                Resource(1),
                Resource(1)]
    G.delayMon = [Monitor(), Monitor(), Monitor()]
    G.numberMon = Monitor()
    
    # simulate
    for i in range(K): # 5 jobs
            job = Job(str(i))
            activate(job, job.run(N))
    simulate(until=maxtime)
    
    # gather performance measures
    W = []
    for i in range(len(G.delayMon)):
        W.append(G.delayMon[i].mean())
    
    L = G.numberMon.timeAverage()

    return W,L

## Experiment -----------------
K = 5

allW = [[],[],[]]
allL = [[],[],[]]
for k in range(50):
    seed = 123*k
    W,L = model(server=100000, N=100000, maxtime=2000000, rvseed=seed)
    for i in range(3):
        allW[i].append(W[i])
        allL[i].append(L)
        
# for question (a)
for i in range(3):
    print("Estimate of W[%d]: %s" % (i, numpy.mean(allW[i])))    
    print("Conf int of W[%d]: %s" % (i, conf(allW[i])))

# for question (b)
print("Estimate of L: ", numpy.mean(allL))    
print("Conf int of L: ", conf(allL))          

Estimate of W[0]: 20.010666247941867
Conf int of W[0]: (19.99390045296098, 20.027432042922754)
Estimate of W[1]: 16.85646455166988
Conf int of W[1]: (16.830563822042365, 16.882365281297396)
Estimate of W[2]: 103.12208361995602
Conf int of W[2]: (102.94568812404994, 103.2984791158621)
Estimate of L:  127.5
Conf int of L:  (45.84966830032674, 209.15033169967325)


## Q2

In [5]:
class Source(Process):
    """generate random arrivals"""
    def run(self, N, lamb):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run())
            t = random.expovariate(lamb)
            yield hold, self, t
            
class Arrival(Process):
    """an arrival"""
    def run(self):
        for i in range(2):
            startTime = now()
            yield request, self, G.server[i]
            t = random.expovariate(G.mu[i])
            yield hold, self, t
            yield release, self, G.server[i]
            delay = now() - startTime
            G.delayMon[i].observe(delay)
        
class G:
    server = 'dummy'
    delayMon = 'Monitor'
    numberMon = 'Monitor'
    startNode = 0
    outNode = 2
    mu = [4.0, 4.0]  
    transition = [
        [1.0, 0, 0],
        [0, 1.0, 0]
    ]    
    
def model(N, lamb, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    # node 1: 1 server; node 2: 2 servers; node 3: 3 servers
    G.server = [
        Resource(2, monitored=True), 
        Resource(2, monitored=True)
    ]
    G.delayMon = [Monitor(), Monitor()]
    
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb))
    simulate(until = maxtime)
    
    # gather performance measures
    W = []
    for i in range(len(G.delayMon)):
        W.append(G.delayMon[i].mean())
    
    L = []
    for i in range(len(G.server)):
        L.append(G.server[i].waitMon.timeAverage()+G.server[i].actMon.timeAverage())

    return W, L

## Experiment
allW = [[],[]]
allL = [[],[]]
allP = [[],[]]
allLambdaEff = []
for k in range(50):
    seed = 123*k
    W, L = model(N=10000, lamb=3, maxtime=2000000, rvseed=seed)
    
    for i in range(2):
        allW[i].append(W[i])
        allL[i].append(L[i])
    
        if (i==1): # <=> section B
            allLambdaEff.append(L[i]/W[i])
    
for i in range(2):
    print("Estimate of W[%d]: %s" % (i, numpy.mean(allW[i])))    
    print("Conf int of W[%d]: %s" % (i, conf(allW[i])))

for i in range(2):
    # average number of customers per section
    avg_num = numpy.mean(allL[i])
    print("Estimate of L[%d]: %s" % (i, avg_num))    
    print("Conf int of L[%d]: %s" % (i, conf(allL[i])))    
    

    print("The proportion of time both servers in the section [%d] are busy: %s" % (i, avg_num/(1+avg_num)))
    
    

print("Estimate of Lambda Eff at section B: ", numpy.mean(allLambdaEff))    
print("Conf int of Lambda Eff at section B: ", conf(allLambdaEff))

Estimate of W[0]: 0.29166679802726775
Conf int of W[0]: (0.2901659827659885, 0.293167613288547)
Estimate of W[1]: 0.2919090947418822
Conf int of W[1]: (0.29044322237899434, 0.29337496710477007)
Estimate of L[0]: 0.8765137951917816
Conf int of L[0]: (0.8710430561422704, 0.8819845342412929)
The proportion of time both servers in the section [0] are busy: 0.4670969099388907
Estimate of L[1]: 0.8772076751915009
Conf int of L[1]: (0.872280876439684, 0.8821344739433178)
The proportion of time both servers in the section [1] are busy: 0.4672938891015421
Estimate of Lambda Eff at section B:  3.0050503626563336
Conf int of Lambda Eff at section B:  (2.9980499350645426, 3.0120507902481246)
